# Shallow Water Equations - With Coriolis

Our model before made a few assumptions (in addition to linearity): no rotation, no shear, and no friction. This gave a simplified version of the shallow water equations. Now, let's relax the irrotational assumption. That is, we will now assume we have a shallow water system sitting somewhere on the rotating Earth. We can of course consider rotations on other planets, but let's focus on our own for now. 

The Coriolis motion will deflect motion to the right in the northern hemisphere, and to the left in the southern hemisphere, by a rate proportional to its velocity. So our tendency equations for $u$ and $v$ will have to have a tendency

$$
\begin{align}
\frac{\partial u}{\partial t} & = \cdots + fv \\
\frac{\partial v}{\partial t} & = \cdots - fu
\end{align}
$$

where $f$ is "Coriolis parameter" that proportionally deflects the flow. In order for it to deflect to the right in the northern hemisphere where the latitude $\phi > 0$, and to the left in the southern hemisphere where $\phi < 0$, we need $f$ to be a function of $\phi$ such that

$$
\phi > 0 \iff f(\phi) > 0  \\
\phi < 0 \iff f(\phi) < 0 .
$$

And indeed, the Coriolis parameter is given as 

$$
f = 2 \Omega \sin \phi.
$$

The term $\Omega$ is the rotational rate of the Earth, which is 

$$
\Omega = \frac{2 \pi}{24 \cdot 60 \cdot 60 \text{ s}} \approx 7.27 \times 10^{-5}.
$$

Since $\phi$ is a function of $y$, $f$ is therefore also a function of $y$, and we need to find a way to handle this One method is to assign the Coriolis acceleration using a constant $f = f_0$. Here, we assume a constant rate of rotation to get the $f$-plane Coriolis parameter:

$$
f_0 = 2 \Omega \sin \phi_0.
$$

For this approximation, we provide a reference latitude $\phi_0$. This $f$-plane approximation can be visualized as a cylinder with radius $a \cos \phi$ where $a$ is the radius of the Earth. In this visualization, the rotational effect of the Coriolis acceleration would be limited to the outer surface of this cylinder such that no matter what latitude the fluid moves to, it still experiences the exact same Coriolis parameter. A better approximation is to let $f$ vary slightly with $y$, using a first order Taylor series approximation:

$$
f \approx f \big|_{\phi_0} + \frac{\mathrm{d}f}{\mathrm{d}y} \big|_{\phi_0} (\phi - \phi_0) = f_0 + 2 \Omega \cos \phi_0 (\phi - \phi_0) = f_0 + \frac{2 \Omega \cos \phi_0}{a} y = f_0 + \beta y
$$

This approximation is the "$\beta$-plane" approximation. Whereas the $f$-plane approximation can be visualized by a cylinder that intersects the earth at latitude $\phi_0$, the $\beta$-plane approximation can be visualized by an infinitely tall cone that sits tangent to the planet's latitude circle at latitude $\phi_0$.

I mentioned before that we can consider rotation on other planets. We are now prepared to do that. All we would need to do is calculate a new rotation rate $\Omega$ which is just 

$$
\Omega = \frac{2 \pi}{T}
$$

where $T$ is how many seconds are in a single rotation, or one day, on that planet. Additionally, for our $\beta$-plane approximation, we would need to replace the Earth's radius $a$ with the radius of this other planet we might wish to consider. For now, I will stick to a shallow water system on Earth. 

So now our model becomes

$$
\begin{align}
\frac{\partial \eta}{\partial t} & = -\frac{\partial (h u^\prime)}{\partial x} - \frac{\partial (h v^\prime)}{\partial y} \\
\frac{\partial u}{\partial t} & = - g \frac{\partial \eta}{\partial x}  + fv\\
\frac{\partial v}{\partial t} & = - g \frac{\partial \eta}{\partial y} - fu .
\end{align}
$$

Just like before, we'll discretize this system of equations and update future time steps via

$$
\begin{align}
\eta^{n+1}_{i,j}& = \eta^{n}_{i,j} - \Delta t \Big( \frac{h^n_{i+1,j} u^{n}_{i+1,j} - h^n_{i,j} u^{n}_{i,j} }{\Delta x} -\frac{h^n_{i,j+1} v^{n}_{i,j+1} - h^n_{i,j} v^{n}_{i,j}}{\Delta y} \Big)  \\
u^{n+1}_{i,j} & = u^{n}_{i,j} - g \Delta t \frac{h^n_{i+1,j} - h^n_{i,j}}{\Delta x} + f v^{n}_{i,j} \\
v^{n+1}_{i,j} & = v^{n}_{i,j} - g \Delta t \frac{h^n_{i,j+1} - h^n_{i,j}}{\Delta y} - f u^{n}_{i,j}.
\end{align}
$$

## Import Modules

Import the modules we need, mainly for `numpy` and `matplotlib`. I've also defined some custom functions in `viz_utils.py` that provide plotting utility.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib import animation
import viz_utils

## Physical Parameters

Now let's set the physical parameters of the model.

In [ ]:
# Domain length in x and y directions [m]
Lx = 1000000.
Ly = 1000000.

# Acceleration of gravity [m/s^2]
g = 9.81

# Average depth of fluid [m]
H = 100.

## Computational Parameters

And now for the computational paramters, such as the number of gridpoints and timesteps.

In [ ]:
# Number of grid points in x and y directions and time steps
nx = 150
ny = 150
nt = 5000

# Grid spacing and time step (defined from CFL criteria)
dx = Lx/(nx - 1)
dy = Ly/(ny - 1)
dt = 0.1*min(dx, dy)/np.sqrt(g*H)

# Define grid
x = np.linspace(-Lx/2, Lx/2, nx)
y = np.linspace(-Ly/2, Ly/2, ny)
X, Y = np.meshgrid(x, y)
X = np.transpose(X)
Y = np.transpose(Y)

## Initialize Arrays

Set initial arrays to zeros before making calculations.

In [ ]:
# Initialize arrays for present (n) and future (n+1)
u_present = np.zeros((nx,ny))
u_future = np.zeros((nx,ny))
v_present = np.zeros((nx,ny))
v_future = np.zeros((nx,ny))
eta_present = np.zeros((nx,ny))
eta_future = np.zeros((nx,ny))

# Temporary variables (each time step) for upwind scheme in eta equation
h_e = np.zeros((nx,ny))
h_w = np.zeros((nx,ny))
h_n = np.zeros((nx,ny))
h_s = np.zeros((nx,ny))
uh_we = np.zeros((nx,ny))
vh_ns = np.zeros((nx,ny))

## Initial Conditions

Now let's specify the initial conditions.

In [ ]:
# Initial conditions for u and v
u_present[:, :] = 0.0
v_present[:, :] = 0.0

# Ensure boundary conditions are met
u_present[-1, :] = 0.0
v_present[:, -1] = 0.0

# Set initial disturbance for eta
eta_present = np.exp(-((X-250000)**2/(2*(0.05E+6)**2) + (Y-250000)**2/(2*(0.05E+6)**2)))

And now we plot the initial conditions.

In [ ]:
viz_utils.contour_plots(x,y,u_present,v_present,eta_present+H,0)

In [ ]:
surface_plot(X,Y,H+eta_present,0,H-1,H+1)

## Initialize Sampling Lists

When we run the model, we'll calculate everything, store values we want to plot, and then go back and plot afterwards. We create those lists and initial values here.

In [ ]:
# Set up sampling variables as empty lists
eta_list,u_list,v_list = [],[],[]
hov_sample, timeseries_sample, time_sample = [],[],[]

# Append initial values to list
hov_sample.append(eta_present[:, int(ny/2)])
timeseries_sample.append(eta_present[int(nx/2), int(ny/2)])
time_sample.append(0.0)

# Set sampling intervals
animation_interval = 20
sample_interval = 1000

## Intergrate the Model Over All Timesteps

And now to integrate the model to predict $u$, $v$, and $h$. We start by updating $u$ and $v$, and then use these values to update $h$. We also enforce the upwind scheme by using the `np.where` function to make sure that we're taking the proper differencing direction based on the flow at that grid point. 

In [ ]:
for n in range(nt):

    # Update u and v for next timestep
    u_future[:-1, :] = u_present[:-1, :] - g*dt/dx*(eta_present[1:, :] - eta_present[:-1, :])
    v_future[:, :-1] = v_present[:, :-1] - g*dt/dy*(eta_present[:, 1:] - eta_present[:, :-1])
    
    # Enforce boundary conditions
    v_future[:, -1] = 0.0
    u_future[-1, :] = 0.0

    # Use upwind scheme needed to update eta for eastward flow
    h_e[:-1, :] = np.where(u_future[:-1, :] > 0, eta_present[:-1, :] + H, eta_present[1:, :] + H)
    h_e[-1, :] = eta_present[-1, :] + H

    # Use upwind scheme needed to update eta for westward flow
    h_w[0, :] = eta_present[0, :] + H
    h_w[1:, :] = np.where(u_future[:-1, :] > 0, eta_present[:-1, :] + H, eta_present[1:, :] + H)

    # Use upwind scheme needed to update eta for northward flow
    h_n[:, :-1] = np.where(v_future[:, :-1] > 0, eta_present[:, :-1] + H, eta_present[:, 1:] + H)
    h_n[:, -1] = eta_present[:, -1] + H

    # Use upwind scheme needed to update eta for southward flow
    h_s[:, 0] = eta_present[:, 0] + H
    h_s[:, 1:] = np.where(v_future[:, :-1] > 0, eta_present[:, :-1] + H, eta_present[:, 1:] + H)

    # Use upwind scheme for quantity u*h
    uh_we[0, :] = u_future[0, :]*h_e[0, :]
    uh_we[1:, :] = u_future[1:, :]*h_e[1:, :] - u_future[:-1, :]*h_w[1:, :]

    # Use upwind scheme for quantity v*h
    vh_ns[:, 0] = v_future[:, 0]*h_n[:, 0]
    vh_ns[:, 1:] = v_future[:, 1:]*h_n[:, 1:] - v_future[:, :-1]*h_s[:, 1:]

    # Update eta for next time step
    eta_future[:, :] = eta_present[:, :] - dt*(uh_we[:, :]/dx + vh_ns[:, :]/dy)

    # Update to next time step
    u_present = np.copy(u_future)
    v_present = np.copy(v_future)
    eta_present = np.copy(eta_future)

    # Samples for Hovmuller diagram and spectrum every sample_interval time step.
    if ((n+1) % sample_interval == 0):
        hov_sample.append(eta_present[:, int(ny/2)])
        timeseries_sample.append(eta_present[int(nx/2), int(ny/2)])
        time_sample.append((n+1)*dt)

    # Store eta and (u, v) every anin_interval time step for animations.
    if ((n+1) % animation_interval == 0):
        u_list.append(u_present)
        v_list.append(v_present)
        eta_list.append(eta_present)

In [ ]:
eta_animation(X,Y,eta_list,dt,"eta_contour",sample_interval)

In [ ]:
quiver_animation(X,Y,u_list,v_list,dt,"flow_vector",sample_interval,stride=4)

In [ ]:
surface_animation(X,Y,eta_list,H,dt,'h_surf',sample_interval)